In [8]:
import stata_setup
import pandas as pd
import numpy as np
import tempfile
from scipy.stats import norm
import os
#stata_setup.config("C:/Program Files/StataNow19/", "be")
stata_setup.config("/Applications/StataNow", "mp")
print("Stata setup complete.")
from pystata import stata

Stata setup complete.


In [9]:

def run_leebounds(df):
    # Ensure correct types
    df['Y'] = df['Y'].astype(float)
    df['D'] = df['D'].astype(int)

    # Transfer the dataframe to Stata (in memory)
    stata.pdataframe_to_data(df, force=True)

    # Diagnostic: Check missing Y
    stata.run('count if missing(Y)', quietly=True)

    # Run Lee bounds
    stata.run('leebounds Y D')


In [10]:
# Create treatment
D = np.random.binomial(1, 0.5, 1000)

# Simulate outcome
Y = 5 + 2 * D + np.random.normal(0, 1, 1000)

# Attrition: treated group has more dropouts
p_s = np.where(D == 1, 0.7, 0.9)
S = np.random.binomial(1, p_s)

# Apply attrition: make Y missing if S==0
Y_observed = np.where(S == 1, Y, np.nan)

# Create DataFrame
df = pd.DataFrame({'Y': Y_observed, 'D': D, 'S': S})

# Run Lee bounds
run_leebounds(df)



Lee (2009) treatment effect bounds

Number of obs.                     =   1000
Number of selected obs.            =   804
Trimming porportion                =   0.2027

------------------------------------------------------------------------------
           Y | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
D            |
       lower |    1.67745    .079684    21.05   0.000     1.521272    1.833628
       upper |   2.348952   .0795541    29.53   0.000     2.193029    2.504876
------------------------------------------------------------------------------


In [11]:
print(df['S'].value_counts())


S
1    804
0    196
Name: count, dtype: int64
